In [14]:
pip install pyspellchecker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 9.9 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import pandas as pd
from spellchecker import SpellChecker

# A dictionary of known synonyms or forced corrections
SYNONYMS = {
    "appl": "apple",
    "appin": "apple",
    "aplin": "apple",
    "appln": "apple",
    "aapl": "apple",
    "nvda": "nvidia",
    # Add as many forced mappings as you need
}

def load_domain_dictionary(csv_file_path, text_column='ner_text_cleaned'):
    df = pd.read_csv(csv_file_path)
    texts = df[text_column].dropna()
    
    all_words = []
    for line in texts:
        line_clean = re.sub(r'[^\w\s]', '', line.lower())
        words = line_clean.split()
        all_words.extend(words)
    
    return all_words

def build_spellchecker(csv_file_path):
    # Increase distance to 3 for more flexible corrections (optional)
    spell = SpellChecker(distance=3)

    domain_words = load_domain_dictionary(csv_file_path)
    spell.word_frequency.load_words(domain_words)

    # Boost stock-related words; large frequency ensures they outrank common English words
    stock_names = ["apple", "tesla", "nvidia", "amazon", "google", "meta", "microsoft"]
    for name in stock_names:
        spell.word_frequency.add(name, 1000000)  # Very high frequency to overshadow common words
    
    return spell

def correct_query(spell, user_query):
    query_clean = re.sub(r'[^\w\s]', '', user_query.lower())
    query_words = query_clean.split()
    
    corrected_words = []
    for w in query_words:
        # 1) Force known synonyms first
        if w in SYNONYMS:
            corrected_words.append(SYNONYMS[w])
        # 2) If recognized, keep as-is
        elif w in spell:
            corrected_words.append(w)
        else:
            # 3) Otherwise, use spell.correction
            correction = spell.correction(w)
            corrected_words.append(correction if correction else w)
    
    return " ".join(corrected_words)

def search_engine_demo():
    csv_file_path = "NER_with_sentiment.csv"
    spell = build_spellchecker(csv_file_path)
    
    user_query = input("Enter search query: ")
    suggestion = correct_query(spell, user_query)
    
    if suggestion != user_query.lower():
        print(f"Did you mean: {suggestion}?")
    else:
        print("Your query looks good!")

if __name__ == "__main__":
    search_engine_demo()


/var/folders/d9/99zh55td5557wrk09bgvss_w0000gn/T/ipykernel_2001/516262375.py:17: DtypeWarning: Columns (7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)


Did you mean: samsung galaxy?
